In [11]:
import cv2
from depthai import Pipeline, Device

# Function to detect AprilTags in the frame
def detect_apriltags(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect AprilTags in the frame
    corners, ids, _ = cv2.aruco.detectMarkers(gray, cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250))

    return corners, ids

# Function to capture video using DepthAI and track objects with markers (e.g., AprilTags)
def track_with_marker():
    # Define the DepthAI pipeline
    pipeline = Pipeline()

    # Define the camera node
    cam_rgb = pipeline.createColorCamera()
    cam_rgb.setPreviewSize(300, 300)  # Set preview size

    # Create output queue for the camera node
    cam_out = pipeline.createXLinkOut()
    cam_out.setStreamName("cam_out")
    cam_rgb.preview.link(cam_out.input)

    try:
        # Connect to DepthAI device
        with Device(pipeline) as device:
            # Start pipeline
            device.startPipeline()

            # Get the output queue
            cam_queue = device.getOutputQueue(name="cam_out", maxSize=1, blocking=False)

            # Create a video window
            cv2.namedWindow("AprilTag Tracking", cv2.WINDOW_NORMAL)

            while True:
                # Get the next frame from the output queue
                frame = cam_queue.get().getCvFrame()

                # Detect AprilTags in the frame
                corners, ids = detect_apriltags(frame)

                # Draw detected AprilTags on the frame
                if ids is not None:
                    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

                # Display the frame with AprilTag markers
                cv2.imshow("AprilTag Tracking", frame)

                # Check for 'q' key press to exit the loop
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    except Exception as e:
        print("An error occurred:", e)

    finally:
        cv2.destroyAllWindows()

# Function to capture video using DepthAI and track objects without markers
def track_without_marker():
    # Define the DepthAI pipeline
    pipeline = Pipeline()

    # Define the camera node
    cam_rgb = pipeline.createColorCamera()
    cam_rgb.setPreviewSize(300, 300)  # Set preview size

    # Add YOLO object detection node
    yolo_model = pipeline.createYoloDetection()
    yolo_model.setConfidenceThreshold(0.5)  # Set confidence threshold

    # Link camera to YOLO object detection
    cam_rgb.preview.link(yolo_model.input)

    # Create output queue for the YOLO object detection node
    yolo_out = pipeline.createXLinkOut()
    yolo_out.setStreamName("yolo_out")
    yolo_model.out.link(yolo_out.input)

    try:
        # Connect to DepthAI device
        with Device(pipeline) as device:
            # Start pipeline
            device.startPipeline()

            # Get the output queue
            yolo_queue = device.getOutputQueue(name="yolo_out", maxSize=1, blocking=False)

            # Create a video window
            cv2.namedWindow("Object Tracking", cv2.WINDOW_NORMAL)

            while True:
                # Get the next frame from the output queue
                frame = yolo_queue.get().getCvFrame()

                # Display the frame with detected objects
                cv2.imshow("Object Tracking", frame)

                # Check for 'q' key press to exit the loop
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    except Exception as e:
        print("An error occurred:", e)

    finally:
        cv2.destroyAllWindows()

# Main function to choose between tracking with or without marker
if __name__ == "__main__":
    method = input("Choose tracking method (1: with marker, 2: without marker): ")

    if method == "1":
        track_with_marker()
    elif method == "2":
        track_without_marker()
    else:
        print("Invalid choice. Choose either 1 or 2.")


Choose tracking method (1: with marker, 2: without marker):  2


AttributeError: 'depthai.Pipeline' object has no attribute 'createYoloDetection'